# Practice 1

## Download the dataset and copy to drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
!wget https://storage.hpai.bsc.es/mame-dataset/MAMe_data_256.zip
!wget https://storage.hpai.bsc.es/mame-dataset/MAMe_metadata.zip
#!cp MAMe_metadata.zip /content/drive/MyDrive/MAI/DL/
#!cp -rv MAMe_data_256.zip /content/drive/MyDrive/MAI/DL/


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
--2022-03-29 14:59:19--  https://storage.hpai.bsc.es/mame-dataset/MAMe_data_256.zip
Resolving storage.hpai.bsc.es (storage.hpai.bsc.es)... 84.88.186.2
Connecting to storage.hpai.bsc.es (storage.hpai.bsc.es)|84.88.186.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 723471982 (690M) [application/zip]
Saving to: ‘MAMe_data_256.zip.1’

MAMe_data_256.zip.1 100%[===================>] 689.96M  10.9MB/s    in 71s     

2022-03-29 15:00:30 (9.69 MB/s) - ‘MAMe_data_256.zip.1’ saved [723471982/723471982]

--2022-03-29 15:00:30--  https://storage.hpai.bsc.es/mame-dataset/MAMe_metadata.zip
Resolving storage.hpai.bsc.es (storage.hpai.bsc.es)... 84.88.186.2
Connecting to storage.hpai.bsc.es (storage.hpai.bsc.es)|84.88.186.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 868837 (848K) [application/zip]
Saving to: ‘MAMe

In [2]:
#!cp /content/drive/MyDrive/MAI/DL/MAMe_metadata.zip MAMe_metadata.zip
#!cp /content/drive/MyDrive/MAI/DL/MAMe_data_256.zip MAMe_data_256.zip
!unzip MAMe_data_256.zip
!unzip MAMe_metadata.zip

Archive:  MAMe_data_256.zip
replace data_256/100019.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data_256/100033.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  MAMe_metadata.zip
replace MAMe_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Imports

In [1]:
import pandas as pd
import numpy as np
from skimage import io


## Prepare Data

In [2]:
file_labels = pd.read_csv("MAMe_dataset.csv").to_numpy()
train = file_labels[file_labels[:, 4] == "train"]
#test = file_labels[file_labels[:,4] == "test"]
validation = file_labels[file_labels[:,4] == "val"]

print("Train size: ", len(train),  "\nValidation size: ", len(validation))

Train size:  20300 
Validation size:  1450


In [3]:
names_images_train = train[:,0]
#names_images_test = test[:,0]
names_images_val = validation[:,0]
y_train = train[:,1]
#y_test = test[:,1]
y_val = validation[:,1]
del train, validation, file_labels

In [4]:
x_train, x_val = [], []
for name in names_images_train:
  x_train.append(io.imread('/content/data_256/'+ name))
x_train = np.array(x_train)

for name in names_images_val:
  x_val.append(io.imread('/content/data_256/'+name))
x_val = np.array(x_val)
 

In [5]:
NUM_CLASSES = 29
def labels_to_dict(labels):
    my_labels = {}
    for i in range(len(labels)):
        my_labels[labels[i,1]] = labels[i,0]
    return my_labels

def load_labels():
    """
    Load labels from csv file
    @return numpy array of labels
    """
    labels = pd.read_csv("MAMe_labels.csv", header=None).to_numpy()
    return labels_to_dict(labels)

def transform_labels(y_train, y_val):
  labels_dict = load_labels()
  y_train_labels, y_val_labels = [], []
  for x in y_train:
        y = np.zeros(NUM_CLASSES)
        y[labels_dict[x]] = 1
        y_train_labels.append(y)
  y_train = np.array(y_train_labels)
  del y_train_labels
  for x in y_val:
        y = np.zeros(NUM_CLASSES)
        y[labels_dict[x]] = 1
        y_val_labels.append(y)
  y_val = np.array(y_val_labels)
  del y_val_labels
  return y_train, y_val


y_train, y_val = transform_labels(y_train, y_val)


In [6]:
print(y_train[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]


# Defining Model

In [7]:
import tensorflow as tf
from tensorflow import keras
from keras import layers



In [8]:

model = keras.Sequential()
model.add(layers.Conv2D(8, kernel_size=(5, 5), activation='gelu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='gelu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='gelu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='gelu'))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 252, 252, 8)       608       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 8)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 64)      4672      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 128)      0

In [9]:
from keras.preprocessing.image import ImageDataGenerator

model.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(lr=0.01),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])
val_data_gen_args = dict(rescale = None,samplewise_center=True,samplewise_std_normalization=True)
val_datagen = ImageDataGenerator(val_data_gen_args)
val_set = val_datagen.flow(x_val, y_val, batch_size=32)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
mdl_fit = model.fit(x_train, y_train, validation_data=val_set, shuffle=True,epochs=60, batch_size=128)

Epoch 1/60
159/159 [==============================] - ETA: 0s - loss: 41.9420 - accuracy: 0.9533

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


159/159 [==============================] - 117s 659ms/step - loss: 41.9420 - accuracy: 0.9533 - val_loss: 0.1587 - val_accuracy: 0.9654
Epoch 2/60
159/159 [==============================] - 102s 641ms/step - loss: 0.1552 - accuracy: 0.9655 - val_loss: 0.1595 - val_accuracy: 0.9654
Epoch 3/60
159/159 [==============================] - 102s 642ms/step - loss: 0.1534 - accuracy: 0.9656 - val_loss: 0.1590 - val_accuracy: 0.9655
Epoch 4/60
159/159 [==============================] - 102s 642ms/step - loss: 0.1546 - accuracy: 0.9656 - val_loss: 0.1632 - val_accuracy: 0.9654
Epoch 5/60
159/159 [==============================] - 102s 642ms/step - loss: 0.1514 - accuracy: 0.9656 - val_loss: 0.1625 - val_accuracy: 0.9654
Epoch 6/60
159/159 [==============================] - 102s 642ms/step - loss: 0.1504 - accuracy: 0.9657 - val_loss: 0.1621 - val_accuracy: 0.9654
Epoch 7/60
159/159 [==============================] - 102s 642ms/step - loss: 0.1501 - accuracy: 0.9658 - val_loss: 0.1607 - val_accur

In [ ]:
plt.plot(mdl_fit.history['loss'], label='train loss')
plt.plot(mdl_fit.history['val_loss'], label='val loss')
plt.legend()
plt.savefig("data_epochs_" +str(60)+"_batchSize_"+ str(32)+'_LossVal_loss')
plt.show()
# plot the accuracy
plt.plot(mdl_fit.history['accuracy'], label='train acc')
plt.plot(mdl_fit.history['val_accuracy'], label='val acc')
plt.legend()
plt.savefig("balanced_data_epochs_" +str(60)+"_batchSize_"+ str(32)+'_AccVal_acc')
#save model to disk
plt.show()